### Primary objectives:
1. Score / weigh effectiveness of each intervention for various countries using a weighted combination of scoring methods
2. Assign a daily aggregated intervention score for each country using the calculated intervention weights - these scores will be used for case count projection

In [1]:
import sys
sys.path.insert(1, 'src')
import config
import intervention_effectiveness_scorer as intv_scorer

import warnings
warnings.simplefilter('ignore')
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import urllib.request
import os
from datetime import datetime, timedelta
from matplotlib import pyplot

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Fetch latest intervention data for countries

We used the publicly available data from https://oxcgrtportal.azurewebsites.net/api/CSVDownload for our experiments. But feel free to use any other more granular data following a similar structure.

In [7]:
# Set this flag to True if you want to download the latest COVID19 intervention data from respective web source
# Set it as False in case of subsequent runs on the same day.
LOAD_LATEST_DATA = True


In [3]:
if LOAD_LATEST_DATA:
    url = config.oxcgrt_intervention_data_online
    local_file = os.path.join(config.base_data_dir, config.oxcgrt_intervention_data_offline)
    try:
        with urllib.request.urlopen(url) as response, open(local_file, 'wb') as out_file:
            data = response.read() # a `bytes` object
            out_file.write(data)
            print ('Downloaded latest data from: {}'.format(url))
    except Exception as e:
        print ('Error while downloading {}: {}'.format(url, e.__class__))    

Downloaded latest data from: https://oxcgrtportal.azurewebsites.net/api/CSVDownload


### Derive the Effectiveness Score for different interventions 

In [4]:
intv_scorer.data_src = os.path.join(config.base_data_dir, config.oxcgrt_intervention_data_offline)
#Select a country only if it has exceeded the conf_cases_threshold
intv_scorer.conf_cases_threshold = 10000
#Select records having confirmed cases >= min_case_threshold
intv_scorer.min_case_threshold = 0
#window for rollong averages of conf case counts
intv_scorer.smoothing_window_len = 3
#number of lags to use for time-series style modeling of conf cases
intv_scorer.num_lags = 1
#Skip a few recent dayes data for potential missing values
intv_scorer.recent_days_to_skip = 5 
#median incubation period for Covid19
intv_scorer.incubation_period = 5

fit_stringency_index = 0.5
fit_conf_cases = 0.5
fit_intv_effect = 0.0
intv_scorer.intervention_scoring_methods = {'fit_stringency_index':fit_stringency_index, 
                                'fit_conf_cases':fit_conf_cases, 
                                'fit_intv_effect':fit_intv_effect}

#Export location of intervention scores
analysis_outcome_export_loc = os.path.join(config.base_data_dir, config.intervention_impacts_loc)
#Export location of weighted & aggregated intervention scores
aggregated_intervention_scores_export_loc = os.path.join(config.base_data_dir, config.intervention_scores_loc)

### Score relative weights of individual interventions for each country

In [5]:
#selected_countries = ['IND', 'USA', 'GBR', 'ITA', 'JPN', 'SGP', 'NLD', 'ISR', 'BEL', 'BRA', 'DEU', 'CUB', 'ESP', 'MEX', 'MYS', 'PHL', 'HUN', 'ZAF']
selected_countries = None
# Calculating relative weights/importance of different interventions
data_all, selected_countries, all_country_intv_scores = intv_scorer.score_interventions (selected_countries=None)
all_country_intv_scores.to_csv(analysis_outcome_export_loc, index=False)


Countries with more than 10000 confirmed cases: 81
* * * * * * * * * * Scoring interventions for country: Afghanistan [AFG]
Data dimension for counry AFG is (165, 17)


,country_code,country_name,intervention,score_stringency_idx,score_conf_cases,score_intv_effects,score_final
0,AFG,Afghanistan,C8_International travel controls,0.844501,1.000000,NA,0.922250
1,AFG,Afghanistan,C7_Restrictions on internal movement,1.000000,0.154313,NA,0.577156
2,AFG,Afghanistan,C4_Restrictions on gatherings,0.981037,0.127937,NA,0.554487
3,AFG,Afghanistan,C1_School closing,0.854614,0.063362,NA,0.458988
4,AFG,Afghanistan,C5_Close public transport,0.748420,0.106260,NA,0.427340
5,AFG,Afghanistan,C2_Workplace closing,0.706700,0.097317,NA,0.402009
6,AFG,Afghanistan,C3_Cancel public events,0.734513,0.012733,NA,0.373623
7,AFG,Afghanistan,C6_Stay at home requirements,0.575221,0.062907,NA,0.319064
8,AFG,Afghanistan,H1_Public information campaigns,0.000000,0.000000,NA,0.000000


* * * * * * * * * * Scoring interventions for country: United Arab Emirates [ARE]
Data dimension for counry ARE is (194, 17)


,country_code,country_name,intervention,score_stringency_idx,score_conf_cases,score_intv_effects,score_final
0,ARE,United Arab Emirates,C6_Stay at home requirements,1.000000,0.731159,NA,0.865580
1,ARE,United Arab Emirates,C2_Workplace closing,0.549020,1.000000,NA,0.774510
2,ARE,United Arab Emirates,C3_Cancel public events,0.537525,0.364685,NA,0.451105
3,ARE,United Arab Emirates,C8_International travel controls,0.609195,0.222907,NA,0.416051
4,ARE,United Arab Emirates,C5_Close public transport,0.473969,0.275403,NA,0.374686
5,ARE,United Arab Emirates,C1_School closing,0.494929,0.195267,NA,0.345098
6,ARE,United Arab Emirates,C4_Restrictions on gatherings,0.342799,0.191291,NA,0.267045
7,ARE,United Arab Emirates,C7_Restrictions on internal movement,0.365450,0.132432,NA,0.248941
8,ARE,United Arab Emirates,H2_Testing policy,0.292765,0.100616,NA,0.196691
9,ARE,United Arab Emirates,H1_Public information campaigns,0.251521,0.028833,NA,0.140177


* * * * * * * * * * Scoring interventions for country: Argentina [ARG]
Data dimension for counry ARG is (220, 17)


...

,country_code,country_name,intervention,score_stringency_idx,score_conf_cases,score_intv_effects,score_final
0,UZB,Uzbekistan,C2_Workplace closing,0.877317,0.983085,NA,0.930201
1,UZB,Uzbekistan,C4_Restrictions on gatherings,1.000000,0.828823,NA,0.914411
2,UZB,Uzbekistan,C1_School closing,0.671227,1.000000,NA,0.835613
3,UZB,Uzbekistan,C8_International travel controls,0.732127,0.539242,NA,0.635685
4,UZB,Uzbekistan,C6_Stay at home requirements,0.374669,0.531800,NA,0.453234
5,UZB,Uzbekistan,C5_Close public transport,0.427184,0.326116,NA,0.376650
6,UZB,Uzbekistan,C7_Restrictions on internal movement,0.414828,0.276049,NA,0.345438
7,UZB,Uzbekistan,H1_Public information campaigns,0.398058,0.118403,NA,0.258231
8,UZB,Uzbekistan,H2_Testing policy,0.135481,0.000000,NA,0.067741
9,UZB,Uzbekistan,H3_Contact tracing,0.039276,0.000000,NA,0.019638


* * * * * * * * * * Scoring interventions for country: Venezuela [VEN]
Data dimension for counry VEN is (220, 17)


,country_code,country_name,intervention,score_stringency_idx,score_conf_cases,score_intv_effects,score_final
0,VEN,Venezuela,C5_Close public transport,0.766504,1.000000,NA,0.883252
1,VEN,Venezuela,C4_Restrictions on gatherings,1.000000,0.373279,NA,0.686639
2,VEN,Venezuela,C2_Workplace closing,0.443765,0.902091,NA,0.672928
3,VEN,Venezuela,C6_Stay at home requirements,0.714548,0.117287,NA,0.415917
4,VEN,Venezuela,C1_School closing,0.316015,0.239674,NA,0.277844
5,VEN,Venezuela,H1_Public information campaigns,0.422983,0.012239,NA,0.217611
6,VEN,Venezuela,C8_International travel controls,0.279951,0.074962,NA,0.177456
7,VEN,Venezuela,C3_Cancel public events,0.305012,0.000000,NA,0.152506
8,VEN,Venezuela,H2_Testing policy,0.000000,0.278429,NA,0.139215
9,VEN,Venezuela,C7_Restrictions on internal movement,0.062958,0.054054,NA,0.058506


* * * * * * * * * * Scoring interventions for country: South Africa [ZAF]
Data dimension for counry ZAF is (220, 17)


,country_code,country_name,intervention,score_stringency_idx,score_conf_cases,score_intv_effects,score_final
0,ZAF,South Africa,C6_Stay at home requirements,0.774002,1.000000,NA,0.887001
1,ZAF,South Africa,C1_School closing,1.000000,0.456686,NA,0.728343
2,ZAF,South Africa,C2_Workplace closing,0.830636,0.224558,NA,0.527597
3,ZAF,South Africa,C8_International travel controls,0.491370,0.045416,NA,0.268393
4,ZAF,South Africa,C3_Cancel public events,0.403452,0.128680,NA,0.266066
5,ZAF,South Africa,H2_Testing policy,0.251348,0.275862,NA,0.263605
6,ZAF,South Africa,H1_Public information campaigns,0.515642,0.009532,NA,0.262587
7,ZAF,South Africa,C7_Restrictions on internal movement,0.396440,0.028595,NA,0.212518
8,ZAF,South Africa,E1_Income support,0.000000,0.177740,NA,0.088870
9,ZAF,South Africa,C5_Close public transport,0.106796,0.026913,NA,0.066855


In [6]:
interventions = all_country_intv_scores['intervention'].unique().tolist()
relevant_cols = ['CountryName', 'CountryCode', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex'] + interventions

data_filtered = data_all.loc[data_all['CountryCode'].isin(selected_countries), relevant_cols].copy()
data_filtered.reset_index(inplace=True)
data_filtered.fillna(0, inplace=True)
# Assign an aggregated intervention score for each country, each day
data_filtered = intv_scorer.assign_weighted_aggregations (data_filtered, all_country_intv_scores, selected_countries)

print ('* '*50 + '\nComparing the Stringency Index of the data provider (OxCGRT) and our calculated Aggregated Intervention Scores:')
display (data_filtered.groupby('CountryName')[['StringencyIndex', 'aggr_weighted_intv_norm']].aggregate(['mean', 'median', 'std']).sort_values(by=('StringencyIndex', 'median'), ascending=False).style.background_gradient(cmap='PuBu'))
data_filtered.to_csv(aggregated_intervention_scores_export_loc)


* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
Comparing the Stringency Index of the data provider (OxCGRT) and our calculated Aggregated Intervention Scores:
